In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_959444/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.4)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_43_1,0.995325,0.641765,0.582998,0.995064,0.001578,0.212664,0.292118,0.001785,0.017951,0.039727,0.990576,0.040036,142.902882,329.292621,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
1,model_1_43_0,0.995304,0.642717,0.585252,0.995460,0.001585,0.212099,0.290539,0.001642,0.017917,0.039816,0.990534,0.040126,142.893947,329.283686,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
2,model_1_43_2,0.995265,0.638576,0.577461,0.993874,0.001598,0.214557,0.295997,0.002215,0.018075,0.039978,0.990457,0.040289,142.877726,329.267465,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
3,model_1_43_3,0.995262,0.638474,0.577359,0.993834,0.001599,0.214617,0.296069,0.002230,0.018080,0.039992,0.990450,0.040303,142.876277,329.266016,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
4,model_1_43_4,0.995243,0.637857,0.575989,0.993576,0.001606,0.214984,0.297028,0.002323,0.018156,0.040071,0.990412,0.040383,142.868407,329.258147,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,model_1_0_4,0.393804,0.279932,0.266683,0.486898,0.204633,0.427464,0.513704,0.106546,0.240518,0.452364,-0.221863,0.455884,133.173076,319.562815,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
496,model_1_0_3,0.323020,0.275434,0.238097,0.363882,0.228528,0.430134,0.533729,0.132090,0.243210,0.478046,-0.364538,0.481766,132.952197,319.341936,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
497,model_1_0_2,0.227021,0.152822,0.145732,0.220849,0.260934,0.502921,0.598433,0.161791,0.277312,0.510817,-0.558035,0.514792,132.686978,319.076717,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
498,model_1_0_1,0.221546,0.106609,0.104542,0.366357,0.262782,0.530355,0.627287,0.131576,0.276563,0.512623,-0.569072,0.516612,132.672860,319.062599,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
